<a href="https://colab.research.google.com/github/Gutobastos/imersao-iA-3-Alura/blob/main/projeto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip -q install google-genai
%pip -q install google-adk

In [1]:
import os

# E necessário inserir a chave na variável do seu ambiente com sua GOOGLE_API_KEY
google_api_key = os.environ.get('GOOGLE_API_KEY')
if google_api_key:
    print("Chave da API do Google encontrada nas variáveis de ambiente do seu PC.\n")
else:
    print("Chave da API do Google não encontrada nas variáveis de ambiente e tentaremos pelo GOOGLE COLAB.\n")
    from google.colab import userdata
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

Chave da API do Google encontrada nas variáveis de ambiente do seu PC.



In [2]:
from google import genai
client = genai.Client()
modelo = "gemini-2.0-flash"

In [3]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
from datetime import date
import textwrap
from IPython.display import display, Markdown
import requests
import warnings

warnings.filterwarnings("ignore")

In [4]:
def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [5]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [6]:
##########################################
# --- Agente 1: BUSCADOR --- #
##########################################
def agente_buscador(topico, data_de_hoje):
  buscador = Agent(
      name = "agente_buscador",
      model = "gemini-2.0-flash",
      description = "Agente que busca informações no Google",
      tools = [google_search],
      instruction = """
      Você é um assistente de pesquisa de móveis e deve usar a ferramenta de 
      busca do Google (google_search), procure pelo móvel e que tenha preço e sejam o mais barato,
      faça uma planilha de preços no final. 
      Que tenha garantia e boa referência de reputação e avaliação a partir da data de hoje.
      Somente guarde resultados que possuem preços.
      """
  )
  entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

  moveis = call_agent(buscador, entrada_do_agente_buscador)
  return moveis

In [7]:
################################################
# --- Agente 2: ORÇAMENTO --- #
################################################
def agente_cotador(topico, moveis_buscados):
    planejador = Agent(
        name="agente_cotacao",
        model="gemini-2.0-flash",
        description="Agente cotação de preços",
        tools=[google_search],
        instruction="""
        Você é um filtro de pesquisa, especialista em encontrar o móvel com melhor custo 
        benefícioa partir da data de hoje. Você deve usar a feramenta de busca do Google (google_search) e dar 
        prioridade preço mais baixo.
        Escolha de 1 até 5 móveis que tenham valores ou preços e com formas de pagamentos se houver.
        """
    )
    entrada_do_agente_cotador = f"Tópico:{topico}\nLançamentos buscados: {moveis_buscados}"

    moveis_do_post = call_agent(planejador, entrada_do_agente_cotador)
    return moveis_do_post

In [8]:
######################################
# --- Agente 3: AVALIADOR --- #
######################################
def agente_avaliador(topico, avaliacao_de_post):
    redator = Agent(
        name="agente_avaliador",
        model="gemini-2.5-pro-preview-03-25",
        description="Agente avaliador de posts",
        instruction="""
            Você é um avaliador de controle de qualidade, você deve priorizar 
            as médias de avaliações, você deve usar a feramenta de busca do Google (google_search).
            Guarde os preços e inclua na tabela.
            Escolha até 5 produtos que tenha a melhor média de avaliação e 3 melhores mostrando em tabela.
            comentários de clientes se tiver.
            """
    )
    entrada_do_agente_avaliador = f"Tópico: {topico}\nPlano de post: {avaliacao_de_post}"

    rascunho = call_agent(redator, entrada_do_agente_avaliador)
    return rascunho

In [15]:
##########################################
# --- Agente 4: RESULTADO --- #
##########################################
def agente_retorno(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.5-pro-preview-03-25",
        description="Agente revisor de post para redes sociais.",
        instruction="""
            Você é um vendedor e dará informação da mais completa sobre o produto, 
            você é especializado vendas varejista, com foco no melhor móvel, com melhor custo, benefício e com menor prazo de entrega se tiver.
            Por ter um público, entre 20 e 50 anos, use um tom de escrita adequado, respeitoso e confiante.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho avaliado, se estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            Mostre uma tabelha com até 5 melhores opções e que tenham preço em ordem crescente, com o nome das empresas e suas avaliações positivas.
            Faça hiperlinks de cada produto da pesquisa inserido na planilha.
            """
    )
    entrada_do_agente_retorno = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"

    texto_final = call_agent(revisor, entrada_do_agente_retorno)
    return texto_final

In [ ]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("📜 Iniciando o Sistema de Pesquisa de Preços de móveis ou produtos com avaliação de qualidade com 4 Agentes IA do Google 📜\n")
while topico != "sair" or topico != "SAIR" or topico != "Sair":
  topico = input("🔍 Por favor, digite o tipo do Móvel ou Produto desejado sobre o qual você quer encontrar ou comprar 🔍: ")
  if topico == "sair" or topico == "SAIR" or topico == "Sair":
    print("O programa foi encerrado com sucesso.")
    break

  if not topico:
    print("🤔 Você não digitou um tipo de móvel ou produto. Por favor, tente novamente ou digite SAIR para finalizar o programa. 🤔\n")
  else :
    print(f"💲 Pronto, vamos buscar pelo eu móvel ou produto e exibir o orçamento 💲: {topico}" )

    moveis_buscados = agente_buscador(topico, data_de_hoje)
    cotador_de_custos = agente_buscador (topico, moveis_buscados)
    avaliador_de_empresas = agente_buscador (topico, cotador_de_custos)
    resultado_do_post = agente_buscador (topico, avaliador_de_empresas)
      
    print("\n----------------- RESULTADO DOS AGENTES -----------------\n")
    display(to_markdown(resultado_do_post))
    print("----------------------------------------------------------------")

📜 Iniciando o Sistema de Pesquisa de Preços de móveis ou produtos com avaliação de qualidade com 4 Agentes IA do Google 📜
💲 Pronto, vamos buscar pelo eu móvel ou produto e exibir o orçamento 💲: lápis de cor

----------------- RESULTADO DOS AGENTES -----------------



> Com base nas informações mais recentes e considerando a data de hoje (17 de maio de 2025), compilei uma tabela atualizada com opções de lápis de cor, seus preços aproximados e considerações sobre reputação e garantia.
> 
> **Tabela de Lápis de Cor (Maio de 2025)**
> 
> | Marca/Modelo                                                                  | Preço (aproximado) | Reputação/Avaliação                                                                                                                                  | Garantia                                                                                                                                                                                                                                     | Onde Comprar                                                                                                                                |
> | :---------------------------------------------------------------------------- | :----------------- | :---------------------------------------------------------------------------------------------------------------------------------------------------- | :----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | :------------------------------------------------------------------------------------------------------------------------------------------ |
> | Faber-Castell EcoLápis Triangular Jumbo (12/24 cores)                         | R$ 32,99 - R$ 40,40    | Boa para uso escolar, formato ergonômico para crianças.                                                                                                | Verificar no ponto de venda (geralmente garantia contra defeitos de fabricação).                                                                                                                                                | Shopee, Mercado Livre                                                                                                                     |
> | Staedtler Noris Colour 185 (24 cores)                                         | R$ 35,99 - R$ 41,29   | Durável, resistente à quebra, bom para uso escolar. Design ergonômico.                                                                    | Verificar no ponto de venda (geralmente garantia contra defeitos de fabricação).                                                                                                                                                | Shopee, Lojas de departamento                                                                                                                             |
> | Faber-Castell SuperSoft (Avulso)                                              | R$ 6,99              | Cores vibrantes, boa cobertura. Mina macia que desliza facilmente pelo papel.                                                               | Verificar no ponto de venda (geralmente garantia contra defeitos de fabricação).                                                                                                                                                | Shopee, Kalunga                                                                                                                                                           |
> | Maped Color Peps Infinity (18 cores)                                           | R$ 19,99 - R$ 24,89             | Variedade de cores, pode ter diferentes acabamentos (neon, metalizado, pastel). Design Jumbo, mais fácil de segurar.                            | Verificar no ponto de venda (geralmente garantia contra defeitos de fabricação).                                                                                                                                                | Shopee, Lojas de departamento                                                                                                                                    |
> | Leo & Leo (48 cores)                                                         | R$ 22,99 - R$ 29,99   | Opção mais acessível.                                                                                                                          | Garantia do vendedor (geralmente 3 meses).                                                                                                                                                                                     | Shopee, Lojas de departamento                                                                                                                                            |
> 
> **Considerações Adicionais:**
> 
> *   **Marcas Recomendadas:** Faber-Castell, Prismacolor, Caran d'Ache, Derwent, Staedtler são frequentemente citadas como marcas de boa qualidade. A Faber-Castell é considerada uma das melhores e mais tradicionais marcas.
> *   **Tipos de Lápis:** Existem lápis de cor tradicionais, aquareláveis e com diferentes tipos de mina (macia, dura). A escolha depende do uso pretendido. Para uso escolar, marcas como Faber-Castell, Staedtler e Crayola são recomendadas por serem duráveis e acessíveis. Para desenhos realistas, Faber-Castell Polychromos, Caran d'Ache Luminance e Prismacolor Premier são boas opções.
> *   **Onde Comprar:** Shopee e Mercado Livre são opções para encontrar diversas marcas e ofertas. Kalunga também é uma opção.
> *   **Garantia:** A garantia pode variar dependendo do fabricante e do varejista. É importante verificar as políticas de garantia antes de efetuar a compra. Em geral, as lojas oferecem garantia contra defeitos de fabricação. Algumas marcas oferecem garantia estendida.
> 
> **Observações:**
> 
> *   Os preços podem variar dependendo do varejista e da região.
> *   A disponibilidade de produtos e modelos pode mudar ao longo do tempo.
> *   Recomenda-se verificar as avaliações mais recentes de outros consumidores antes de tomar uma decisão de compra.
> *   Para artistas amadores que buscam qualidade média e uma ampla gama de cores, a marca Castle Arts é uma boa opção. Outras alternativas mais acessíveis aos Prismacolor incluem Faber Castell Black Edition e Star Joy Gold.


----------------------------------------------------------------
💲 Pronto, vamos buscar pelo eu móvel ou produto e exibir o orçamento 💲: cadeira para escritório
